In [2]:
# import spacy
# import en_core_web_sm
# nlp = en_core_web_sm.load()

import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
import csv

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from ecig_parsing import *


In [3]:
def get_html(url, clicked=False, closed=False, elements=True):
    # print('GET', url)
    try:
        driver = webdriver.Firefox()
        driver.get(url)
        last_n = 0
        hrefs = set()
        button_id = 'submit_birthdate'

        html = driver.page_source
        time.sleep(1)
        same_count = 0
        if not clicked:
            wait = WebDriverWait(driver, 10)  # 10 seconds timeout
            button = wait.until(EC.element_to_be_clickable((By.ID, button_id)))  # Use ID, XPATH, or other locator

            # Click the button
            button.click()
            clicked = True
        if not closed:
            wait = WebDriverWait(driver, 10)  # 10 seconds timeout
            button = wait.until(EC.element_to_be_clickable((By.ID, 'omnisend-form-65006a9819ed2c118d9a9352-close-action')))  # Use ID, XPATH, or other locator

            # Click the button
            button.click()
            closed = True
    

        # Set a delay for dynamic content loading
        SCROLL_PAUSE_TIME = 0.5  # Shorter pause for smoother scroll

        # Scroll incrementally by a small step (e.g., 500 pixels)
        scroll_increment = 500

        # Get the initial scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        first_height = last_height
        # print(last_height)

        while True:
            # Scroll down by the increment
            driver.execute_script(f"window.scrollBy(0, {scroll_increment});")

            # Wait for the page to load new content
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate the new scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            # Check if new elements have appeared on the screen
            # You can check specific elements like this:
            # elements = driver.find_elements(By.CLASS_NAME, "your-element-class")

            # If the scroll height has not changed, break the loop
            #print(same_count, new_height, last_height)
            if new_height == last_height :
                same_count += 1
                if same_count >= 3:
                    break
            else:
                same_count = 0

            # Update the last height
            last_height = new_height
            
            html = driver.page_source
            #print(html)
        

    except Exception as ex:
        print(ex)
        html = ''
    finally:
        if driver:
            driver.close()




    return html



In [4]:
links = ['https://vape.com/collections/starter-kits',
         'https://vape.com/collections/devices',
         'https://vape.com/collections/tanks',
         'https://vape.com/collections/accessories',
         'https://vape.com/collections/e-liquids',
         'https://vape.com/collections/alternatives',
         'https://vape.com/collections/disposable-e-cigs',
         'https://vape.com/collections/clearance',
    '/collections/vandy-vape',
 '/collections/vandy-vape',
 '/collections/voopoo',
 '/collections/voopoo',
 '/collections/uwell',
 '/collections/uwell',
 '/collections/freemax',
 '/collections/freemax',
 '/collections/hellvape',
 '/collections/hellvape',
 '/collections/snowwolf',
 '/collections/snowwolf',
 '/collections/naked-100',
 '/collections/naked-100',
 'https://vape.com/collections/7-daze',
 'https://vape.com/collections/acrohm-fush',
 'https://vape.com/collections/adam-bomb',
 'https://vape.com/collections/a-final-course',
 'https://vape.com/collections/air-factory',
 'https://vape.com/collections/al-fakher',
 'https://vape.com/collections/alien-vape',
 'https://vape.com/collections/aloha-sun',
 'https://vape.com/collections/alpha',
 'https://vape.com/collections/alt-bar',
 'https://vape.com/collections/alt-zero',
 'https://vape.com/collections/Anarchist',
 'https://vape.com/collections/aqua-e-juice',
 'https://vape.com/collections/aspire-vape',
 'https://vape.com/collections/augvape',
 'https://vape.com/collections/bad-drip',
 'https://vape.com/collections/badger-hill-reserve',
 'https://vape.com/collections/bams-cannoli',
 'https://vape.com/collections/bang-vape',
 'https://vape.com/collections/banger',
 'https://vape.com/collections/bar-juice',
 'https://vape.com/collections/barz',
 'https://vape.com/collections/basix',
 'https://vape.com/collections/bazooka',
 'https://vape.com/collections/bb-tank',
 'https://vape.com/collections/beach-club',
 'https://vape.com/collections/beard-vape-co',
 'https://vape.com/collections/beast',
 'https://vape.com/collections/big-bar',
 'https://vape.com/collections/big-boy',
 'https://vape.com/collections/black-out-delta-8',
 'https://vape.com/collections/blank-bar',

 'https://vape.com/collections/blessed',
 'https://vape.com/collections/blvd',
 'https://vape.com/collections/blvk',
 'https://vape.com/collections/bolt',
 'https://vape.com/collections/boulder',
 'https://vape.com/collections/brella-salts',
 'https://vape.com/collections/brewell-mfg',
 'https://vape.com/collections/bruce-pro-innovations',
 'https://vape.com/collections/buckshot-vapors',
 'https://vape.com/collections/bugler',
 'https://vape.com/collections/burst',
 'https://vape.com/collections/cali-alternatives',
 'https://vape.com/collections/cali-bars',
 'https://vape.com/collections/candy-king',
 'https://vape.com/collections/cartel',
 'https://vape.com/collections/ccell',
 'https://vape.com/collections/cereal-treats',
 'https://vape.com/collections/chain-vapez',
 'https://vape.com/collections/chapo-extrax',
 'https://vape.com/collections/charco-blaze',
 'https://vape.com/collections/charge',
 'https://vape.com/collections/charlies-chalk-dust',
 'https://vape.com/collections/chill',
 'https://vape.com/collections/chillax',
 'https://vape.com/collections/chubby-bubble-vapes',
 'https://vape.com/collections/chubby-gorilla',
 'https://vape.com/collections/cleanaf',
 'https://vape.com/collections/cloud-nurdz',
 'https://vape.com/collections/clutch-vapors',
 'https://vape.com/collections/coastal-clouds',
 'https://vape.com/collections/coilmaster',
 'https://vape.com/collections/congrevape',
 'https://vape.com/collections/cookie-king',
 'https://vape.com/collections/council-of-vapor',
 'https://vape.com/collections/creme-de-la-creme',
 'https://vape.com/collections/crown-bar',
 'https://vape.com/collections/crushed',
 'https://vape.com/collections/culinary-confections',
 'https://vape.com/collections/custard-monster',
 'https://vape.com/collections/custard-shoppe',
 'https://vape.com/collections/cuttwood-vapors',
 'https://vape.com/collections/czar',
 'https://vape.com/collections/dab-rite',

 'https://vape.com/collections/dairy-king',
 'https://vape.com/collections/delta-extrax',
 'https://vape.com/collections/delta-8-hemp',
 'https://vape.com/collections/dimo',
 'https://vape.com/collections/dinner-lady',
 'https://vape.com/collections/directors-cut',
 'https://vape.com/collections/dji',
 'https://vape.com/collections/donuts',
 'https://vape.com/collections/dotmod',
 'https://vape.com/collections/dovpo',
 'https://vape.com/collections/dozo',
 'https://vape.com/collections/dragbar',
 'https://vape.com/collections/dream-switch',
 'https://vape.com/collections/drip-bar',
 'https://vape.com/collections/dripmore',
 'https://vape.com/collections/dr-shugar-chitz',
 'https://vape.com/collections/drip-stix',
 'https://vape.com/collections/drippin',
 'https://vape.com/collections/drop-plus',
 'https://vape.com/collections/dual',

 'https://vape.com/collections/duo',
 'https://vape.com/collections/dutch',
 'https://vape.com/collections/ebdesign',
 'https://vape.com/collections/efest',
 'https://vape.com/collections/ehpro',
 'https://vape.com/collections/eighty-six',
 'https://vape.com/collections/eleaf',
 'https://vape.com/collections/elf-bar',
 'https://vape.com/collections/elysian-labs',
 'https://vape.com/collections/enou',
 'https://vape.com/collections/erth',
 'https://vape.com/collections/eos',
 'https://vape.com/collections/esco-bars',
 'https://vape.com/collections/evo',
 'https://vape.com/collections/evod',
 'https://vape.com/collections/evolve',

 'https://vape.com/collections/ezzy',
 'https://vape.com/collections/fair-grounds',
 'https://vape.com/collections/fire-disposable',
 'https://vape.com/collections/flawless',
 'https://vape.com/collections/flum',
 'https://vape.com/collections/focus-v-carta',
 'https://vape.com/collections/fog-x-vapors',
 'https://vape.com/collections/food-fighter-juice',
 'https://vape.com/collections/four-seasons',
 'https://vape.com/collections/freemax',
 'https://vape.com/collections/fresca',
 'https://vape.com/collections/fresh-farms',
 'https://vape.com/collections/fryd',
 'https://vape.com/collections/fume',
 'https://vape.com/collections/fuze',
 'https://vape.com/collections/g-o-a-t',
 'https://vape.com/collections/geek-bar',
 'https://vape.com/collections/geekvape',
 'https://vape.com/collections/glamee',
 'https://vape.com/collections/glas-vapor',

 'https://vape.com/collections/gost-vapors',
 'https://vape.com/collections/got',
 'https://vape.com/collections/hale',
 'https://vape.com/collections/halo',
 'https://vape.com/collections/hangsen',
 'https://vape.com/collections/helixbar',
 'https://vape.com/collections/hellvape',
 'https://vape.com/collections/hi-drip',

 'https://vape.com/collections/hitt',
 'https://vape.com/collections/hohm-tech',
 'https://vape.com/collections/holy-cannoli',
 'https://vape.com/collections/horizontech',
 'https://vape.com/collections/humble-juice-co',
 'https://vape.com/collections/hyppe',
 'https://vape.com/collections/ignite',
 'https://vape.com/collections/ijoy',
 'https://vape.com/collections/infzn',
 'https://vape.com/collections/innevape',
 'https://vape.com/collections/iqs',
 'https://vape.com/collections/jamba',
 'https://vape.com/collections/jazzy-boba',
 'https://vape.com/collections/johnny-applevapes',
 'https://vape.com/collections/johnny',
 'https://vape.com/collections/joyetech',
 'https://vape.com/collections/juice-head',
 'https://vape.com/collections/juice-house',
 'https://vape.com/collections/juice-man',
 'https://vape.com/collections/juice-roll-upz',
 'https://vape.com/collections/juicy-af',
 'https://vape.com/collections/juul',
 'https://vape.com/collections/kangertech',
 'https://vape.com/collections/kangvape',
 'https://vape.com/collections/keep-it-100',
 'https://vape.com/collections/khali-vapors',
 'https://vape.com/collections/killa-fruits',
 'https://vape.com/collections/kilo-e-liquid',
 'https://vape.com/collections/koou',
 'https://vape.com/collections/krush-it',
 'https://vape.com/collections/lemon-life',
 'https://vape.com/collections/limitless',
 'https://vape.com/collections/liquid-efx',
 'https://vape.com/collections/lollidrip',
 'https://vape.com/collections/lolly-mod',
 'https://vape.com/collections/loose-leaf',
 'https://vape.com/collections/lost-art',
 'https://vape.com/collections/lost-mary',
 'https://vape.com/collections/lost-vape',
 'https://vape.com/collections/loud',
 'https://vape.com/collections/lucid',
 'https://vape.com/collections/lush',

 'https://vape.com/collections/mad-hatter',
 'https://vape.com/collections/mega',
 'https://vape.com/collections/mi-pod',
 'https://vape.com/collections/mighty-vapors-salt-e-liquid',
 'https://vape.com/collections/mints',
 'https://vape.com/collections/mlv',
 'https://vape.com/collections/mnke',
 'https://vape.com/collections/monster-vape-labs',
 'https://vape.com/collections/moti',
 'https://vape.com/collections/mr-freeze',
 'https://vape.com/collections/mrkt-plce',
 'https://vape.com/collections/mucho-ejuice',
 'https://vape.com/collections/myle',
 'https://vape.com/collections/naked-100',
 'https://vape.com/collections/nasty-juice',
 'https://vape.com/collections/nic-fill',
 'https://vape.com/collections/nitecore',
 'https://vape.com/collections/nitros-cold-brew',
 'https://vape.com/collections/nomenon',
 'https://vape.com/collections/nupop',
 'https://vape.com/collections/okvmi',
 'https://vape.com/collections/omg-liquids',
 'https://vape.com/collections/one-hit-wonder',
 'https://vape.com/collections/one-up',
 'https://vape.com/collections/ooze',
 'https://vape.com/collections/orgnx',
 'https://vape.com/collections/oro-twist',
 'https://vape.com/collections/oxva',
 'https://vape.com/collections/onevape',
 'https://vape.com/collections/pachamama-e-liquid',

 'https://vape.com/collections/pax-labs',
 'https://vape.com/collections/phyre',
 'https://vape.com/collections/pod-juice',
 'https://vape.com/collections/pop-clouds',
 'https://vape.com/collections/premium-vibez',
 'https://vape.com/collections/propaganda-e-liquid',
 'https://vape.com/collections/puff',
 'https://vape.com/collections/puff-labs',
 'https://vape.com/collections/puffco',
 'https://vape.com/collections/pyrovape',
 'https://vape.com/collections/ram',
 'https://vape.com/collections/reds-tfn-salt',

 'https://vape.com/collections/riot-squad',
 'https://vape.com/collections/runtz-wraps',
 'https://vape.com/collections/ruthless',
 'https://vape.com/collections/ryse',
 'https://vape.com/collections/sadboy',
 'https://vape.com/collections/saveurvape',
 'https://vape.com/collections/send-it',
 'https://vape.com/collections/sense',
 'https://vape.com/collections/shijin-vapor',
 'https://vape.com/collections/sigelei',
 'https://vape.com/collections/silverback-juice',
 'https://vape.com/collections/skol',
 'https://vape.com/collections/skwezed',
 'https://vape.com/collections/slammin',
 'https://vape.com/collections/slapwoods',
 'https://vape.com/collections/smoktech',
 'https://vape.com/collections/snap-liquids',
 'https://vape.com/collections/snowwolf',
 'https://vape.com/collections/soc',
 'https://vape.com/collections/solace',
 'https://vape.com/collections/stache-products',

 'https://vape.com/collections/steam-engine',
 'https://vape.com/collections/strange-fruit',
 'https://vape.com/collections/strio',
 'https://vape.com/collections/stumps',
 'https://vape.com/collections/sugoi-vapor',
 'https://vape.com/collections/suicide-bunny',
 'https://vape.com/collections/sunvape',
 'https://vape.com/collections/suorin',
 'https://vape.com/collections/swft',
 'https://vape.com/collections/taffy-fiend',
 'https://vape.com/collections/tartz',
 'https://vape.com/collections/taylor',
 'https://vape.com/collections/the-mamasan',
 'https://vape.com/collections/thirsty',
 'https://vape.com/collections/time-bomb-vapors',
 'https://vape.com/collections/tinted-brew',

 'https://vape.com/collections/to-the-max',
 'https://vape.com/collections/tobeco',
 'https://vape.com/collections/topshine',
 'https://vape.com/collections/truly-bar',
 'https://vape.com/collections/truweigh',
 'https://vape.com/collections/tucano',
 'https://vape.com/collections/tuglyfe-e-liquids',
 'https://vape.com/collections/tugpod',
 'https://vape.com/collections/twist-e-liquid',
 'https://vape.com/collections/tyson',
 'https://vape.com/collections/twist-zero2',
 'https://vape.com/collections/usa-vape-labs',
 'https://vape.com/collections/uwell',
 'https://vape.com/collections/vaal',
 'https://vape.com/collections/vandy-vape',
 'https://vape.com/collections/vape-100',
 'https://vape.com/collections/vape-breakfast-classics',
 'https://vape.com/collections/vape-pink',
 'https://vape.com/collections/vaper-treats',
 'https://vape.com/collections/vapetasia',
 'https://vape.com/collections/vapmod',
 'https://vape.com/collections/vaporesso',
 'https://vape.com/collections/vaptio',
 'https://vape.com/collections/verdict-vapors',
 'https://vape.com/collections/vgod',
 'https://vape.com/collections/viva',
 'https://vape.com/collections/voopoo',
 'https://vape.com/collections/vozol-ark'
 'https://vape.com/collections/vrk',
 'https://vape.com/collections/vyce',
 'https://vahttp://localhost:8888/notebooks/ecig/vapedotcom.ipynb#pe.com/collections/waka',
 'https://vape.com/collections/wake',
 'https://vape.com/collections/wismec',
 'https://vape.com/collections/wizman',
 'https://vape.com/collections/wolf-grinders',
 'https://vape.com/collections/xtra',
 'https://vape.com/collections/yami-vapor',
 'https://vape.com/collections/yocan',
 'https://vape.com/collections/yogi',
 'https://vape.com/collections/zaza',
 'https://vape.com/collections/zen-haus']

In [5]:
import json
import os

# Specify the file path
json_file_path = 'vapedotcom.json'

# Check if the file exists
if os.path.exists(json_file_path):
    # Open and read the JSON file
    with open(json_file_path, 'r') as file:
        try:
            found = json.load(file)
            print("JSON data:", found.keys())
        except json.JSONDecodeError:
            print("Error: The file contains invalid JSON.")
else:
    found = dict()
    print(f"File '{json_file_path}' does not exist.")

JSON data: dict_keys(['https://vape.com/collections/starter-kits?page=1', 'https://vape.com/products/geekvape-t200-aegis-touch-kit-200w', 'https://vape.com/products/smok-mag-18-kit', 'https://vape.com/products/geekvape-l200-aegis-legend-2-kit-200w', 'https://vape.com/products/geekvape-max100-aegis-max-2-100w-kit', 'https://vape.com/products/smok-morph-3-box-mod-kit', 'https://vape.com/products/smok-priv-n19-30w-kit', 'https://vape.com/products/dovpo-lightsaber-pod-mod-kit', 'https://vape.com/products/lost-vape-thelema-mini-kit-45w-ub-lite-tank', 'https://vape.com/products/voopoo-drag-x-80w-pod-mod-kit', 'https://vape.com/products/voopoo-drag-h40-kit', 'https://vape.com/products/voopoo-drag-q-kit', 'https://vape.com/products/smok-ipx-80-pod-kit-80w', 'https://vape.com/products/voopoo-drag-3-tpp-x-kit-177w', 'https://vape.com/products/geekvape-s100-aegis-solo-2-kit-100w', 'https://vape.com/products/geekvape-e100i-aegis-eteno-kit', 'https://vape.com/products/smok-rpm-80-kit-80w-internal-b

In [14]:
from collections import Counter
import csv

def save():
    print('SAVING...')
    with open('vapedotcom_scrape.csv', mode='w') as file:
        # Create a DictWriter object
        writer = csv.DictWriter(file, fieldnames=product_list[0].keys())

        # Write the header (column names)
        writer.writeheader()

        # Write the rows (each dictionary in the data list)
        writer.writerows(product_list)
        
    with open(json_file_path, 'w') as file:
        json.dump(found, file, indent=4) 

product_list = list()
all_headers = list()
header_counter = Counter(all_headers)
product_tags = set()

for link_index,l in enumerate(links):
    page = 1
    while True:

        print('LINK INDEX', link_index, 'OF', len(links), ' -- PAGE', page, )
        if l.startswith('https'):
            main_url = f'{l}?page={page}'
        else:
            main_url = f'https://vape.com{l}?page={page}'

        try:
            if main_url in found:
                req_text = found[main_url]
            else:
                req_text = get_html(main_url)
                found[main_url] = req_text

            soup = BeautifulSoup(req_text)
            prods = soup.find_all('div', {'class': 'product-wrap'})
            
            if len(prods) == 0:
                break
        except Exception as ex:
            print(ex, 'FAILED', main_url)
            break
            
        page += 1
        
        for wrap in prods:
            psoup = wrap.parent
            # Extract product URL
            product_url_tag = psoup.find('a', href=True)
            product_url = product_url_tag['href'] if product_url_tag else None

            # Extract main image URL (handling the case where the image might not be found)
            image_tag = psoup.find('img', class_='lazyautosizes')
            main_image_url = image_tag['data-src'] if image_tag else None

            # Extract current price
            price_tag = psoup.find('span', class_='current_price')
            current_price = price_tag.text if price_tag else None

            # Extract product name
            product_name_tag = psoup.find('span', itemprop='name')
            product_name = product_name_tag.text if product_name_tag else None

            # Extract brand
            brand_tag = psoup.find('span', class_='brand')
            brand = brand_tag.text if brand_tag else None


#             # Print the extracted information
#             print("Product URL:", product_url)
#             print("Main Image URL:", main_image_url)
#             print("Current Price:", current_price)
#             print("Product Name:", product_name)
#             print("Brand:", brand)

                    # Extracting product information
            if product_url.startswith('http'):
                full_link = product_url
            else:
                full_link = f'https://vape.com{product_url}'
                
            tag = full_link.split('/')[-1]
            if tag in product_tags:
                continue
            
            try:
            
                if full_link in found:
                    reqtxt = found[full_link]
                else:
                    # print(full_link)
                    reqtxt = get_html(full_link, elements=False)
                    found[full_link] = reqtxt
            except Exception as ex:
                print(ex, 'FAILED', main_url)
                continue
            
            description = ''
            warning = ''
            package_contents = ''
            ingredients = ''
            key_features = ''
            flavors = ''
            specifications = ''
            performance = ''
            disposable = ''
            why = ''
            preference = ''
            innovation = ''
            enjoyment = ''
            compatibility = ''
            coil = ''
            battery = ''
            puffs = ''
            nicotine = ''
            e_liquid = ''
            prefilled = ''
            devices = ''
            airflow = ''
            heating_element = ''
            operation = ''
            charging = ''
            screen = ''
            links_txt = ''
            colors = ''
            options_txt = ''
            returns = ''
            size = ''
            pods = ''
            hardware = ''
            wattage = ''

            section_strings_raw = {
                'Display Screen': 'screen',
                'Charging': 'charging',
                'Charging Port Info': 'charging',
                'Operation': 'operation',
                'Prefilled Capacity': 'prefilled',
                'Compatibility': 'compatibility',
                'Compatible with': 'compatibility',
                'Ingredients': 'ingredients',
                'INCLUDES': 'package_contents',
                'Kit Includes': 'package_contents',
                "What's Included": 'package_contents',
                'Package Content': 'package_contents',
                'Package Contents': 'package_contents',
                'Package Includes': 'package_contents',
                'Included in package': 'package_contents',
                'Included in the Package': 'package_contents',
                'Included': 'package_contents',
                'Includes': 'package_contents',
                'Contents': 'package_contents',
                "What’s Included": 'package_contents',
                'CALIFORNIA PROPOSITION 65 - Warning': 'warning',
                'CAUTION: this Product Contains Nicotine. Nicotine is an addictive chemical.': 'warning',
                'Warning': 'warning',
                'AVAILABLE OPTIONS': 'key_features',
                'Features and Specifications': 'key_features',
                'Key Features': 'key_features',
                'Features': 'key_features',
                'Flavors': 'flavors',
                'Flavor Selection': 'flavors',
                'Flavor Options': 'flavors',
                'Cutting-Edge Design and Technology:': 'key_features',
                'Superior Versatility and Performance:': 'key_features',
                'Flavor Selection:': 'flavors',
                'WARNING': 'warning',
                'Description': 'description',
                'Product Specifications': 'specifications',
                'Specifications': 'specifications',
                'Available Flavors': 'flavors',
                'Flavor Profiles': 'flavors',
                'Exceptional Flexibility and Performance': 'performance',
                'Disposable': 'disposable',
                'Lost Mary x Elf Bar OS5000 Disposable Features': 'key_features',
                'Tailored to Your Preference': 'preference',
                'Why Geek Bar Skyview Disposable?': 'why',
                'Enjoy My Shisha on the Go': 'enjoyment',
                'The Ultimate Vaping Experience': 'enjoyment',
                'Elevate Your Vaping Experience': 'enjoyment',
                'Why Choose the Juice Head 30K Disposable?': 'why',
                'Why Geek Bar Skyview Disposable?': 'why',
                'Features and Specifications': 'key_features',
                'Hotbox Features': 'key_features',
                'Lost Mary x Elf Bar OS5000 Disposable Features': 'key_features',
                'Experience the Ultimate Vaping Innovation': 'innovation',
                'Flavor Details': 'flavors',
                'Advanced Dual Mesh Coil': 'coil',
                'Coil Specifications': 'coil',
                'Battery Info': 'battery',
                'Puff Count': 'puffs',
                'Max Puffs': 'puffs',
                'Available Nicotine': 'nicotine',
                'Nicotine': 'nicotine',
                'Nicotine Strength': 'nicotine',
                'Battery Capacity': 'battery',
                'E-liquid Capacity': 'e_liquid',
                'Maximum Puffs': 'puffs',
                'Available Flavor Profiles': 'flavors',
                'Prefilled': 'prefilled',
                'Compatible Devices': 'devices',
                'Devices': 'devices',
                'California Proposition 65 Warning': 'warning',
                'Melli Flavors': 'flavors',
                'Airflow': 'airflow',
                'Heating Element': 'heating_element',
                'Quick Links': 'links',
                'Quick Link': 'links',
                'Colors': 'colors',
                'Color': 'colors',
                'Quick links': 'links',
                'Flavor': 'flavors',
                'Size': 'size',
                'Sizes': 'size',
                'Bottle Size': 'size',
                'Bottle Sizes': 'size',
                'Return and Exchange Policy Disclaimer:': 'returns',
                'Returns': 'returns',
                'Pods': 'pods',
                'Pod Info': 'pods',
                'Options': 'options',
                'Option': 'options',
                'Compatible Hardware': 'hardware',
                'Hardware': 'hardware',
                'Wattage Info': 'wattage',
                'Wattage': 'wattage',
                'Coil': 'coil',
                'Coils': 'coil',
                'Available Nicotine': 'nicotine'
            }
            
            functions = [
                str.upper,
                str.lower,
                str.capitalize,
                str.title,
                str.swapcase,
                str.casefold
            ]
            section_strings = dict()
            for k, v in section_strings_raw.items():
                key = k
                section_strings[key] = v
                if key.endswith(':'):
                    key = key.replace(':', '')
                    section_strings[key] = v
                else:
                    key = key.strip() + ':'
                    section_strings[key] = v
                for func in functions:
                    key = func(k)
                    section_strings[key] = v
                    if key.endswith(':'):
                        key = key.replace(':', '')
                        section_strings[key] = v
                    else:
                        key = key.strip() + ':'
                        section_strings[key] = v
                
            
            section_map = {
                'description': description,
                'warning': warning,
                'package_contents': package_contents,
                'ingredients': ingredients,
                'key_features': key_features,
                'flavors': flavors,
                'specifications': specifications,
                'performance': performance,
                'disposable': disposable,
                'why': why,
                'innovation': innovation,
                'preference': preference,
                'enjoyment': enjoyment,
                'compatibility': compatibility,
                'coil': coil,
                'battery': battery,
                'puffs': puffs,
                'nicotine': nicotine,
                'e_liquid': e_liquid,
                'prefilled': prefilled,
                'devices': devices,
                'airflow': airflow,
                'heating_element': heating_element,
                'operation': operation,
                'charging': charging,
                'screen': screen,
                'links': links_txt,
                'colors': colors,
                'returns': returns,
                'options': options_txt,
                'size': size,
                'pods': pods,
                'hardware': hardware,
                'wattage': wattage
            }
            images = [main_image_url]
            bsoup = BeautifulSoup(reqtxt)
            container = bsoup.find('div', {'class': 'shopify-section shopify-section--product-template is-product-main content'})
            if container:
                images_elems = container.find_all('img')

                # Extract and print image URLs from the img tags
                for img in images_elems:
                    # Check if 'src', 'data-src', or 'data-zoom-src' attributes exist
                    img_url = img.get('data-zoom-src') or img.get('data-src') or img.get('src')

                    # Construct the full URL if necessary (since it's missing the protocol)
                    if img_url and img_url.startswith('//'):
                        img_url = 'https:' + img_url
                    images.append(img_url)

                    # print("Image URL:", img_url)

                
                        # Extract product description
                desc_section = container.find('div', {'class': 'description'})

                cur_section = 'description'
                if desc_section:
                    headers = [tag.text for tag in desc_section.find_all(lambda tag: tag.name in ['strong','h1', 'h2', 'h3', 'h4', 'h5'])]
                    stext = desc_section.get_text(separator='\n').split('\n')
                    for s in stext:
                        s = s.strip()
                        if s == '':
                            continue
                        s_no_colon = s.replace(':', '').strip()
                        if s in section_strings:
                            cur_section = section_strings[s]
                        elif s_no_colon in section_strings:
                            cur_section = section_strings[s_no_colon]
                        elif s.lower().startswith('why') and s.endswith('?'):
                            cur_section = 'why'
                        elif 'features:' in s.lower():
                            cur_section = 'key_features'
                        elif s in headers:
                            all_headers.append(s)
                            header_counter = Counter(all_headers)
                            #if header_counter[s] > 5:
                            #    print(s, full_link)
                            cur_section = 'description'
                            section_map[cur_section] += f'{s}\n\n'
                        else:
                            section_map[cur_section] += f'{s}\n'


            
            product_data = {
                'tag': tag,
                "title": product_name,
                'brand': brand,
                "link": full_link,
                "current_price": current_price,
                'image_url': main_image_url,
                'image_urls': list(set(images)),
#                 'flavor_list': flavors,
#                 'nicotine_strengths': nicotine_strengths,
#                 'bottle_sizes': bottle_sizes,
#                 "stock_status": stock_status
            }
            
            num = 0
            for i in product_data['image_urls']:
                if i and i != '':
                    download_image(i, f'{tag}-image{num}', save_dir='vapedotcom_images')
                    num += 1

            
            desc_fields = ''
            for s, v in section_map.items():
                if 'description' not in s:
                    s = s + '_description'
                product_data[s] = v.replace('\xa0', ' ').strip()
                if product_data[s] != '' and 'link' not in s:
                    desc_fields += f'\n{product_data[s]}'

            feats = list()
            #print(desc_fields)
            feat = find_features(desc_fields)
            any_found, puffs_res, nico_res, ml_res, flav_text, dev_text = feat

            if any_found:
                #print(feat)
                feats.append(feat)
            disposable,recharge,battery,mesh,usb,adjustable,found_flavs = features_to_cats(feats)
            product_data['disposable'] = disposable
            product_data['rechargeable'] = recharge
            product_data['battery'] = battery
            product_data['mesh'] = mesh
            product_data['usb'] = usb
            product_data['adjustable'] = adjustable
            
#             print(product_data)

            #print(product_data)

            product_list.append(product_data)
            product_tags.add(tag)
        
            if len(product_list) % 10 == 0:
                rc = (random.choice(product_list))
                print('{')
                for key, value in rc.items():
                    if not value:
                        continue
                    if key == 'image_urls':
                        continue
                    value = value.replace('\n', ' ')
                    if key == 'image_url' or key == 'link' or key == 'title'  or key == 'tag' :
                        print(f"\t{key}: {value}")
                    elif 1 <= len(value) <= 20:
                        print(f"\t{key}: {value}")
                    elif len(value) > 20:
                        print(f"\t{key}: {value[:20]}...")
                print('}')
        
                print(len(product_list), 'ITEMS FOUND')
                print()
                print()
                for item, count in header_counter.most_common(50):
                    if count > 5:
                        print(f"{item}: {count}")
                print()
                print()
                save()


        
#random.choice(product_list)
            

LINK INDEX 0 OF 322  -- PAGE 1
{
	tag: lost-vape-thelema-mini-kit-45w-ub-lite-tank
	title: Lost Vape Thelema Mini Kit 45W (UB Lite Tank)
	brand: Lost Vape
	link: https://vape.com/products/lost-vape-thelema-mini-kit-45w-ub-lite-tank
	current_price: from $19.95
	image_url: https://cdn.shopify.com/s/files/1/0614/0303/7892/products/All-Lost-Vape-Thelema-Mini-Kit-45W-UB-Lite-Tank-1_1600x.png?v=1669828321
	description: The Lost Vape Thelem...
	package_contents_description: 1x Thelema Mini Devi...
	key_features_description: POWER RANGE 5-45W CU...
	links_description: Lost Vape UB Lite Co...
	colors_description: Carbon Fiber Cappucc...
	battery: TRUE
	adjustable: TRUE
}
10 ITEMS FOUND




SAVING...
{
	tag: dovpo-mvp-220w-box-kit
	title: Dovpo MVP 220w Box Kit
	brand: Dovpo
	link: https://vape.com/products/dovpo-mvp-220w-box-kit
	description: The most recent prod...
	package_contents_description: 1x MVP Box Mod Kit 1...
	key_features_description: Dual High-Amp 18650 ...
	links_description: Dovp

{
	tag: geekvape-l200-aegis-legend-2-classic-kit
	title: Geekvape L200 (Aegis Legend 2) Classic Kit
	brand: GeekVape
	link: https://vape.com/products/geekvape-l200-aegis-legend-2-classic-kit
	current_price: $63.99
	description: Geekvape L200 (Aegis...
	package_contents_description: 1x Aegis Legend 2 mo...
	key_features_description: Smaller & lighter Ac...
	links_description: Geekvape T200 (Aegis...
	colors_description: Black Red Rainbow Vo...
	adjustable: TRUE
}
180 ITEMS FOUND


(NEW): 14


SAVING...
{
	tag: voopoo-drag-s-pro-kit-80w
	title: VooPoo Drag S Pro Kit | 80w
	brand: VooPoo
	link: https://vape.com/products/voopoo-drag-s-pro-kit-80w
	description: Voopoo Drag S Pro Ki...
	package_contents_description: Drag S Pro Device TP...
	key_features_description: Size: 125.5 x 28.8 x...
	links_description: VooPoo Argus Air Kit...
	battery: TRUE
	mesh: TRUE
}
190 ITEMS FOUND


(NEW): 15


SAVING...
{
	tag: lost-vape-centaurus-p200-mod
	title: Lost Vape Centaurus P200 Mod
	brand: Lost Vape


{
	tag: dovpo-mvp-220w-box-mod
	title: Dovpo MVP 220w Box Mod
	brand: Dovpo
	link: https://vape.com/products/dovpo-mvp-220w-box-mod
	image_url: https://cdn.shopify.com/s/files/1/0614/0303/7892/products/All-Dovpo-MVP-220W-Box-Mod_1600x.jpg?v=1664559547
	description: The most recent prod...
	package_contents_description: 1x MVP Box Mod 1x Ty...
	key_features_description: Dual High-Amp 18650 ...
	links_description: Dovpo MVV II Box Mod...
	colors_description: Fire Demon Beast Bla...
	battery: TRUE
	usb: TRUE
}
350 ITEMS FOUND


(NEW): 17
Black: 6


SAVING...
LINK INDEX 3 OF 322  -- PAGE 2
{
	tag: vaporesso-itank-2
	title: Vaporesso iTank 2
	brand: Vaporesso
	link: https://vape.com/products/vaporesso-itank-2
	current_price: $25.09
	image_url: https://cdn.shopify.com/s/files/1/0614/0303/7892/products/all_8af903ae-6086-41e8-ae31-cf4992f5471b_1600x.png?v=1680664755
	description: The newly developed ...
	package_contents_description: 1x VAPORESSO iTANK 2...
	key_features_description: 8ml
	link

{
	tag: dovpo-riva-box-kit
	title: Dovpo Riva Box Kit
	brand: Dovpo
	link: https://vape.com/products/dovpo-riva-box-kit
	current_price: from $20.95
	description: Introducing the Dovp...
	package_contents_description: 1x Riva 200 Box Mod ...
	key_features_description: Dual High-Amp 18650 ...
	links_description: Dovpo MVV II Box Mod...
	colors_description: Black/Black Black/Br...
	battery: TRUE
	usb: TRUE
}
520 ITEMS FOUND


(NEW): 29
Black: 6
To ensure your safety and maintain the battery’s life and durability, we recommend you follow these rules:: 6


SAVING...
{
	tag: smok-tfv18-sub-ohm-tank
	title: Smok TFV18 Sub-Ohm Tank
	brand: SMOK
	link: https://vape.com/products/smok-tfv18-sub-ohm-tank
	current_price: from $27.32
	image_url: https://cdn.shopify.com/s/files/1/0614/0303/7892/products/smok-tfv18-sub-ohm-tank-483767_1600x.jpg?v=1648048124
	description: The SMOK TFV18 Sub-O...
	package_contents_description: TFV18 Tank 0.33ohm T...
	key_features_description: 32mm Diameter 7.5mL ...
	l

{
	tag: horizon-sakerz-replacement-glass-1pc
	title: Horizon SAKERZ Replacement Glass | 1pc
	brand: HorizonTech
	link: https://vape.com/products/horizon-sakerz-replacement-glass-1pc
	current_price: $7.10
	image_url: https://cdn.shopify.com/s/files/1/0614/0303/7892/products/horizon-sakerz-replacement-glass-1pc-vapor-empire-627598_1600x.jpg?v=1648047567
	description: Horizon Tech SAKERZ ...
	package_contents_description: 1pc Horizon Sakerz R...
	key_features_description: 5mL Bubble Glass Cap...
	links_description: Horizon SAKERZ Coils...
}
660 ITEMS FOUND


(NEW): 29
To ensure your safety and maintain the battery’s life and durability, we recommend you follow these rules:: 7
Black: 6


SAVING...
{
	tag: geekvape-aegis-solo-kit-100w
	title: GeekVape Aegis Solo Kit 100w
	brand: GeekVape
	link: https://vape.com/products/geekvape-aegis-solo-kit-100w
	description: The GeekVape AEGIS S...
	package_contents_description: 1 AEGIS SOLO 100W Bo...
	key_features_description: Wattage Output Range...


...

Failed to download https://vape.com/cdn/shop/files/ONT040708_5000x.png?v=1712773573: HTTPSConnectionPool(host='vape.com', port=443): Max retries exceeded with url: /cdn/shop/files/ONT040708_5000x.png?v=1712773573 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x137b50c10>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
{
	tag: lost-vape-thelema-mini-kit-45w-ub-lite-pod-tank
	title: Lost Vape Thelema Mini Kit 45W (UB Lite Pod Tank)
	brand: Lost Vape
	link: https://vape.com/products/lost-vape-thelema-mini-kit-45w-ub-lite-pod-tank
	current_price: $32.31
	description: The Lost Vape Thelem...
	package_contents_description: 1x Thelema Mini Pod ...
	key_features_description: Power Range: 5-45W C...
	links_description: Lost Vape UB Lite Co...
	colors_description: Carbon Fiber Cappucc...
	battery: TRUE
	adjustable: TRUE
}
5340 ITEMS FOUND


*NOT FOR USE IN SUB-OHM DEVICES*: 1433
(NEW): 659
CAUTION: This Product Conta

{
	tag: vaporesso-eco-nano-mesh-pod-6ml-2-pack
	title: Vaporesso Eco Nano Mesh Pod 6mL (2-Pack)
	brand: Vaporesso
	link: https://vape.com/products/vaporesso-eco-nano-mesh-pod-6ml-2-pack
	current_price: from $9.84
	description: Introducing the Vapo...
	package_contents_description: x2 Vaporesso Luxe – ...
	key_features_description: 6mL Pod Capacity E-L...
	links_description: Vaporesso Eco Nano K...
	returns_description: Vape.com does not ac...
	options_description: 0.8Ω 1.2Ω 0.6Ω 0.6Ω ...
	mesh: TRUE
}
5360 ITEMS FOUND


*NOT FOR USE IN SUB-OHM DEVICES*: 1443
(NEW): 673
CAUTION: This Product Contains Nicotine. Nicotine is an addictive chemical.: 483
*There are NO returns or exchanges on disposable vape devices for any reason, defective or otherwise.: 193
VG/PG Ratio: 176
*All disposable vape devices are non-returnable and non-exchangeable, whether defective or otherwise.*: 128
Salts of all nicotine products, including disposables, e-liquids, e-cigarettes, pouches, concentrates, gums, pa

LINK INDEX 303 OF 322  -- PAGE 3
LINK INDEX 303 OF 322  -- PAGE 4
LINK INDEX 304 OF 322  -- PAGE 1
LINK INDEX 304 OF 322  -- PAGE 2
LINK INDEX 305 OF 322  -- PAGE 1
LINK INDEX 305 OF 322  -- PAGE 2
LINK INDEX 306 OF 322  -- PAGE 1
{
	tag: oxva-xlim-v2-replacement-pods-2ml-3-pack
	title: OXVA Xlim V2 Replacement Pods - 2mL | 3-Pack
	brand: OXVA
	link: https://vape.com/products/oxva-xlim-v2-replacement-pods-2ml-3-pack
	description: OXVA Xlim V2 Replace...
	package_contents_description: 3 Pack of OXVA Xlim ...
	key_features_description: 2ml Cartridge capaci...
	coil_description: 1.2Ω (10-12W) Integr...
	links_description: OXVA Xlim V2 Kit
}
5390 ITEMS FOUND


*NOT FOR USE IN SUB-OHM DEVICES*: 1444
(NEW): 685
CAUTION: This Product Contains Nicotine. Nicotine is an addictive chemical.: 483
*There are NO returns or exchanges on disposable vape devices for any reason, defective or otherwise.: 195
VG/PG Ratio: 176
*All disposable vape devices are non-returnable and non-exchangeable, whether de

LINK INDEX 306 OF 322  -- PAGE 2
LINK INDEX 307 OF 322  -- PAGE 1
LINK INDEX 307 OF 322  -- PAGE 2
LINK INDEX 308 OF 322  -- PAGE 1
LINK INDEX 308 OF 322  -- PAGE 2
LINK INDEX 308 OF 322  -- PAGE 3
LINK INDEX 308 OF 322  -- PAGE 4
LINK INDEX 309 OF 322  -- PAGE 1
LINK INDEX 310 OF 322  -- PAGE 1
LINK INDEX 310 OF 322  -- PAGE 2
LINK INDEX 311 OF 322  -- PAGE 1
Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//vahttp//localhost%3A8888/notebooks/ecig/vapedotcom.ipynb%23pe.com/collections/waka%3Fpage%3D1&c=UTF-8&d=We%20can%E2%80%99t%20connect%20to%20the%20server%20at%20vahttp.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:832:5
checkReadyState@chrome://remote/content/marionette/navigate.sys.mjs:58:24
onNavigation@chrome://remote/content/marionette/navigate.sys.mjs:343:39
emit@resource://gre/mo

{
	tag: collab-bundle-smok-novo-pro-device-daddy-s-vapor-10ml-juice
	title: Collab Bundle – SMOK Novo Pro Device + Daddy’s Vapor 10mL Juice
	brand: SMOK
	link: https://vape.com/products/collab-bundle-smok-novo-pro-device-daddy-s-vapor-10ml-juice
	description: Elevate your vaping ...
	package_contents_description: x1 Device x1 Salt Li...
	links_description: Collab Bundle – Uwel...
	options_description: Color: Rainbow Flavo...
}
5440 ITEMS FOUND


*NOT FOR USE IN SUB-OHM DEVICES*: 1477
(NEW): 686
CAUTION: This Product Contains Nicotine. Nicotine is an addictive chemical.: 487
*There are NO returns or exchanges on disposable vape devices for any reason, defective or otherwise.: 201
VG/PG Ratio: 176
*All disposable vape devices are non-returnable and non-exchangeable, whether defective or otherwise.*: 128
Salts of all nicotine products, including disposables, e-liquids, e-cigarettes, pouches, concentrates, gums, patches, lozenges, sprays, inhalers, sho’s Cigarettes Cigars Cigarillos, subst

In [15]:
save()


SAVING...
